In [216]:
import os
import sys
from pathlib import Path

# lägg till projektroten (mappen ovanför notebooks/) på sys.path
root_dir = Path().absolute()

if root_dir.parts[-1:] == ('notebooks',):
    root_dir = Path(*root_dir.parts[:-1])

root_dir = str(root_dir) 
print(f"Root dir: {root_dir}")
print("Local environment")

if root_dir not in sys.path:
    sys.path.append(root_dir)
    print(f"Added the following directory to the PYTHONPATH: {root_dir}")

Root dir: c:\Users\Chris\hockey-agent
Local environment


In [217]:
import hopsworks
from config import settings
import requests
import pandas as pd

In [218]:
project = hopsworks.login(
    project=settings.HOPSWORKS_PROJECT,
    api_key_value=settings.HOPSWORKS_API_KEY,
    host = settings.HOPSWORKS_HOST
)


2025-12-20 13:57:49,953 INFO: Closing external client and cleaning up certificates.
2025-12-20 13:57:49,957 INFO: Connection closed.
2025-12-20 13:57:49,960 INFO: Initializing external client
2025-12-20 13:57:49,961 INFO: Base URL: https://eu-west.cloud.hopsworks.ai:443
2025-12-20 13:57:50,725 INFO: Python Engine initialized.

Logged in to project, explore it here https://eu-west.cloud.hopsworks.ai:443/p/3193


In [219]:
from datetime import datetime

def generate_season_ids(start_year=2000):
    current_year = datetime.now().year

    season_ids = []
    for year in range(start_year, current_year+1):
        season_ids.append(f"{year}{year+1}")

    return season_ids

season_ids = generate_season_ids(2000)
season_ids[:5], season_ids[-3:]

(['20002001', '20012002', '20022003', '20032004', '20042005'],
 ['20232024', '20242025', '20252026'])

In [220]:
import requests
import pandas as pd

def fetch_games_from_nhl(season):
    """
    Hämtar alla matcher för en säsong från NHL REST API.
    """
    url = "https://api.nhle.com/stats/rest/en/game"

    params = {
        "cayenneExp": f"gameType=2 and season={season}"
    }

    response = requests.get(url, params=params)
    response.raise_for_status()

    data = response.json()["data"]
    return pd.DataFrame(data)


In [221]:
def fetch_teams():
    url = "https://api.nhle.com/stats/rest/en/team"
    response = requests.get(url)
    response.raise_for_status()

    teams = response.json()["data"]
    df = pd.DataFrame(teams)

    return df[[
        "id",
        "fullName",
        "franchiseId"
    ]]

In [222]:
games_df = fetch_teams()

team_id_to_name = dict(
    zip(games_df["id"], games_df["fullName"])
)


In [223]:
all_teams = []

for season_id in season_ids:
    try:
        print(f"Hämtar säsong {season_id}")
        df_season = fetch_games_from_nhl(season_id)
        all_teams.append(df_season)
    except Exception as e:
        print(f"Misslyckades för {season_id}: {e}")

games_df = pd.concat(all_teams, ignore_index=True)
games_df

Hämtar säsong 20002001
Hämtar säsong 20012002
Hämtar säsong 20022003
Hämtar säsong 20032004
Hämtar säsong 20042005
Hämtar säsong 20052006
Hämtar säsong 20062007
Hämtar säsong 20072008
Hämtar säsong 20082009
Hämtar säsong 20092010
Hämtar säsong 20102011
Hämtar säsong 20112012
Hämtar säsong 20122013
Hämtar säsong 20132014
Hämtar säsong 20142015
Hämtar säsong 20152016
Hämtar säsong 20162017
Hämtar säsong 20172018
Hämtar säsong 20182019
Hämtar säsong 20192020
Hämtar säsong 20202021
Hämtar säsong 20212022
Hämtar säsong 20222023
Hämtar säsong 20232024
Hämtar säsong 20242025
Hämtar säsong 20252026


,id,easternStartTime,gameDate,gameNumber,gameScheduleStateId,gameStateId,gameType,homeScore,homeTeamId,period,season,visitingScore,visitingTeamId
0,2000020001,2000-10-04T19:00:00,2000-10-04,1,1,7,2,2,25,4,20002001,2,21
1,2000020002,2000-10-05T19:00:00,2000-10-05,2,1,7,2,4,6,4,20002001,4,9
2,2000020003,2000-10-05T19:00:00,2000-10-05,3,1,7,2,4,7,3,20002001,2,16
3,2000020004,2000-10-05T19:00:00,2000-10-05,4,1,7,2,6,4,3,20002001,3,23
4,2000020005,2000-10-05T19:00:00,2000-10-05,5,1,7,2,3,20,3,20002001,4,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30217,2025021308,2026-04-16T20:00:00,2026-04-16,1308,1,1,2,0,52,1,20252026,0,28
30218,2025021309,2026-04-16T20:00:00,2026-04-16,1309,1,1,2,0,68,1,20252026,0,19
30219,2025021310,2026-04-16T21:00:00,2026-04-16,1310,1,1,2,0,20,1,20252026,0,26
30220,2025021311,2026-04-16T21:00:00,2026-04-16,1311,1,1,2,0,22,1,20252026,0,23


In [224]:
import re

def to_snake(name: str) -> str:
    # splitta CamelCase till snake_case
    s = re.sub(r"(.)([A-Z][a-z]+)", r"\1_\2", name)
    s = re.sub(r"([a-z0-9])([A-Z])", r"\1_\2", s)
    return s.lower()

In [225]:
games_df = games_df.rename(columns={
    col: to_snake(col) for col in games_df.columns
})

games_df["home_team_name"] = games_df["home_team_id"].map(team_id_to_name)
games_df["away_team_name"] = games_df["visiting_team_id"].map(team_id_to_name)

games_df

,id,eastern_start_time,game_date,game_number,game_schedule_state_id,game_state_id,game_type,home_score,home_team_id,period,season,visiting_score,visiting_team_id,home_team_name,away_team_name
0,2000020001,2000-10-04T19:00:00,2000-10-04,1,1,7,2,2,25,4,20002001,2,21,Dallas Stars,Colorado Avalanche
1,2000020002,2000-10-05T19:00:00,2000-10-05,2,1,7,2,4,6,4,20002001,4,9,Boston Bruins,Ottawa Senators
2,2000020003,2000-10-05T19:00:00,2000-10-05,3,1,7,2,4,7,3,20002001,2,16,Buffalo Sabres,Chicago Blackhawks
3,2000020004,2000-10-05T19:00:00,2000-10-05,4,1,7,2,6,4,3,20002001,3,23,Philadelphia Flyers,Vancouver Canucks
4,2000020005,2000-10-05T19:00:00,2000-10-05,5,1,7,2,3,20,3,20002001,4,17,Calgary Flames,Detroit Red Wings
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30217,2025021308,2026-04-16T20:00:00,2026-04-16,1308,1,1,2,0,52,1,20252026,0,28,Winnipeg Jets,San Jose Sharks
30218,2025021309,2026-04-16T20:00:00,2026-04-16,1309,1,1,2,0,68,1,20252026,0,19,Utah Mammoth,St. Louis Blues
30219,2025021310,2026-04-16T21:00:00,2026-04-16,1310,1,1,2,0,20,1,20252026,0,26,Calgary Flames,Los Angeles Kings
30220,2025021311,2026-04-16T21:00:00,2026-04-16,1311,1,1,2,0,22,1,20252026,0,23,Edmonton Oilers,Vancouver Canucks


In [226]:
fs = project.get_feature_store()

matches_fg = fs.get_or_create_feature_group(
    name="matches",
    description="NHL matches per season since 2000",
    version=1,
    primary_key=["id"]
)

matches_fg.insert(games_df)

Uploading Dataframe: 100.00% |██████████| Rows 30222/30222 | Elapsed Time: 00:15 | Remaining Time: 00:00


Launching job: matches_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://eu-west.cloud.hopsworks.ai:443/p/3193/jobs/named/matches_1_offline_fg_materialization/executions


(Job('matches_1_offline_fg_materialization', 'SPARK'), None)

Here we get the form of the players

In [227]:
PLAYER_GAME_LOG_URL = "https://api.nhle.com/stats/rest/en/skater/summary"

def fetch_player_form_for_season(season_id: str) -> pd.DataFrame:
    """
    Hämtar game-by-game stats för alla spelare för en given säsong.
    
    Args:
        season_id: Säsong i format YYYYYYYY (t.ex. "20232024")
    
    Returns:
        DataFrame med per-match statistik för alla spelare
    """
    params = {
        "isGame": "true",  # Detta ger per-match stats
        "cayenneExp": f"gameTypeId=2 and seasonId={season_id}",
        "limit": -1  # Hämta alla matcher
    }
    
    resp = requests.get(PLAYER_GAME_LOG_URL, params=params, timeout=20)
    resp.raise_for_status()
    
    data = resp.json()
    
    if "data" not in data or not data["data"]:
        return pd.DataFrame()
    
    df = pd.DataFrame(data["data"])
    
    # Säkerställ att seasonId finns
    if "seasonId" not in df.columns:
        df["seasonId"] = season_id
    
    return df

In [228]:
all_players = []

for season_id in [season_ids[-2], season_ids[-1]]: # Only take the two latest seasons. 
    try:
        print(f"Hämtar säsong {season_id}")
        df_season = fetch_player_form_for_season(season_id)
        all_players.append(df_season)
    except Exception as e:
        print(f"Misslyckades för {season_id}: {e}")

players_df = pd.concat(all_players, ignore_index=True)
print(players_df.shape)

Hämtar säsong 20242025
Hämtar säsong 20252026
(20000, 30)


In [229]:
players_df = players_df.rename(columns={
    col: to_snake(col) for col in players_df.columns
})
players_df

,assists,ev_goals,ev_points,faceoff_win_pct,game_date,game_id,game_winning_goals,games_played,goals,home_road,...,pp_points,sh_goals,sh_points,shooting_pct,shoots_catches,shots,skater_full_name,team_abbrev,time_on_ice_per_game,season_id
0,0,1,1,0.5,2025-04-03,2024021199,1,1,1,R,...,0,0,0,0.33333,L,3,Parker Kelly,COL,883.0,20242025
1,0,0,0,NaN,2025-03-09,2024021013,0,1,0,H,...,0,0,0,0.00000,L,1,Cam York,PHI,1253.0,20242025
2,1,0,1,NaN,2024-12-14,2024020475,0,1,0,R,...,0,0,0,0.00000,R,3,Brandt Clarke,LAK,777.0,20242025
3,0,0,0,NaN,2025-03-25,2024021130,0,1,0,H,...,0,0,0,NaN,R,0,Chris Tanev,TOR,1086.0,20242025
4,0,0,0,NaN,2024-11-25,2024020344,0,1,0,R,...,0,0,0,0.00000,L,3,Joel Edmundson,LAK,1074.0,20242025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0,0,0,NaN,2025-12-17,2025020529,0,1,0,R,...,0,0,0,0.00000,R,1,Drew Doughty,LAK,1521.0,20252026
19996,0,0,0,NaN,2025-11-22,2025020345,0,1,0,R,...,0,0,0,0.00000,L,1,Mikko Rantanen,DAL,812.0,20252026
19997,0,0,0,NaN,2025-11-13,2025020270,0,1,0,H,...,0,0,0,0.00000,L,2,Ivan Demidov,MTL,915.0,20252026
19998,0,0,0,NaN,2025-11-08,2025020237,0,1,0,R,...,0,0,0,NaN,L,0,Adam Pelech,NYI,1070.0,20252026


In [230]:
matthews_df = players_df[players_df['skater_full_name'] == 'Auston Matthews']
print(matthews_df)

       assists  ev_goals  ev_points  faceoff_win_pct   game_date     game_id  \
371          1         0          1          0.42105  2025-04-02  2024021194   
890          0         1          1          0.63157  2025-03-15  2024021059   
2034         0         0          0          0.55555  2025-01-09  2024020659   
2435         1         0          1          0.55555  2025-02-08  2024020884   
2750         0         2          2          0.38095  2024-12-04  2024020406   
3793         2         1          3          0.57142  2025-01-04  2024020623   
4290         1         0          1          0.36363  2025-02-28  2024020945   
4801         1         0          1          0.68750  2025-03-27  2024021152   
8383         0         0          0          0.57894  2025-03-13  2024021040   
9303         1         0          0          0.73684  2025-03-29  2024021165   
12287        0         0          0          0.40000  2025-11-11  2025020256   
13062        0         0          0     

In [231]:
fs = project.get_feature_store()

players_form_fg = fs.get_or_create_feature_group(
    name="players_form",
    description="NHL player in game stats, since 2000",
    version=1,
    primary_key=["player_id", "season_id", "game_id"]
)

players_form_fg.insert(players_df)

Uploading Dataframe: 100.00% |██████████| Rows 20000/20000 | Elapsed Time: 00:10 | Remaining Time: 00:00


Launching job: players_form_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://eu-west.cloud.hopsworks.ai:443/p/3193/jobs/named/players_form_1_offline_fg_materialization/executions


(Job('players_form_1_offline_fg_materialization', 'SPARK'), None)

In [232]:
# URL för målvakter
GOALIE_GAME_LOG_URL = "https://api.nhle.com/stats/rest/en/goalie/summary"

def fetch_goalie_form_for_season(season_id: str) -> pd.DataFrame:
    """
    Hämtar game-by-game stats för alla målvakter för en given säsong.
    
    Args:
        season_id: Säsong i format YYYYYYYY (t.ex. "20232024")
    
    Returns:
        DataFrame med per-match statistik för alla målvakter
    """
    params = {
        "isGame": "true",  # Detta ger per-match stats
        "cayenneExp": f"gameTypeId=2 and seasonId={season_id}",
        "limit": -1  # Hämta alla matcher
    }
    
    resp = requests.get(GOALIE_GAME_LOG_URL, params=params, timeout=20)
    resp.raise_for_status()
    
    data = resp.json()
    
    if "data" not in data or not data["data"]:
        return pd.DataFrame()
    
    df = pd.DataFrame(data["data"])
    
    # Säkerställ att seasonId finns
    if "seasonId" not in df.columns:
        df["seasonId"] = season_id
    
    return df

In [235]:
all_players = []

for season_id in [season_ids[-2], season_ids[-1]]: # Only take the two latest seasons. 
    try:
        print(f"Hämtar säsong {season_id}")
        df_season = fetch_goalie_form_for_season(season_id)
        all_players.append(df_season)
    except Exception as e:
        print(f"Misslyckades för {season_id}: {e}")

goalies_form_df = pd.concat(all_players, ignore_index=True)

goalies_form_df = goalies_form_df.rename(columns={
    col: to_snake(col) for col in goalies_form_df.columns
})
goalies_form_df = goalies_form_df.drop(columns=['ties'], errors='ignore')
goalies_form_df

Hämtar säsong 20242025
Hämtar säsong 20252026


,assists,game_date,game_id,games_played,games_started,goalie_full_name,goals,goals_against,goals_against_average,home_road,...,points,save_pct,saves,shoots_catches,shots_against,shutouts,team_abbrev,time_on_ice,wins,season_id
0,0,2025-01-26,2024020793,1,1,Spencer Knight,0,3,3.17460,R,...,0,0.88000,22,L,25,0,FLA,3402,0,20242025
1,0,2025-03-26,2024021140,1,1,Spencer Knight,0,4,4.20437,H,...,0,0.80000,16,L,20,0,CHI,3425,0,20242025
2,0,2025-01-16,2024020711,1,1,Elvis Merzlikins,0,1,1.00000,H,...,0,0.94736,18,L,19,0,CBJ,3600,1,20242025
3,0,2025-04-15,2024021297,1,1,Jordan Binnington,0,1,1.00000,H,...,0,0.95652,22,L,23,0,STL,3600,1,20242025
4,0,2024-11-11,2024020239,1,0,Samuel Montembeault,0,0,0.00000,R,...,0,1.00000,4,L,4,0,MTL,969,1,20242025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3915,0,2025-12-17,2025020531,1,1,Joel Hofer,0,0,0.00000,H,...,0,1.00000,24,L,24,1,STL,3600,1,20252026
3916,0,2025-12-18,2025020535,1,1,Alex Lyon,0,3,3.00668,H,...,0,0.88888,24,L,27,0,BUF,3592,1,20252026
3917,0,2025-12-16,2025020527,1,1,Dustin Wolf,0,5,5.18582,R,...,0,0.80000,20,L,25,0,CGY,3471,0,20252026
3918,0,2025-12-17,2025020530,1,1,Cam Talbot,0,4,4.00000,H,...,0,0.85714,24,L,28,0,DET,3600,0,20252026


In [237]:
# Skapa feature group
fs = project.get_feature_store()
goalies_form_fg = fs.get_or_create_feature_group(
    name="goalies_form",
    description="NHL goalie game-by-game stats",
    version=1,
    primary_key=["goalie_full_name", "season_id", "game_id"]
)
goalies_form_fg.insert(goalies_form_df)

Feature Group created successfully, explore it at 
https://eu-west.cloud.hopsworks.ai:443/p/3193/fs/3140/fg/3161


Uploading Dataframe: 100.00% |██████████| Rows 3920/3920 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: goalies_form_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://eu-west.cloud.hopsworks.ai:443/p/3193/jobs/named/goalies_form_1_offline_fg_materialization/executions


(Job('goalies_form_1_offline_fg_materialization', 'SPARK'), None)